<b><font size=20, color='#A020F0'>hvPlot</font></b>

Hannah Zanowski<br>
11/10/25<br>

#### <span style="color:green">Learning Goals</span>
By the end of this notebook you will
1. Understand the utility of hvplot in a data exploration and visualization pipeline
2. Implement hvPlot in lieu of the standard pandas and xarray .plot() APIs
3. Practice making various interactive plots with hvPlot

#### Resources
[hvPlot Website](https://hvplot.holoviz.org/en/docs/latest/index.html)<br>
[hvPlot User Guide](https://hvplot.holoviz.org/en/docs/latest/user_guide/index.html)<br>
[hvPlot API](https://hvplot.holoviz.org/en/docs/latest/ref/api/index.html)

**Non-exhaustive list of packages that hvPlot leverages (in addition to matplotlib):**<br>
[Bokeh](https://bokeh.org/) (interactive visualizations for web browsers)<br>
[Plotly](https://plotly.com/python/)<br>
[Datashader](https://datashader.org/#) (Visualize large datasets easily)<br>
[HoloViews](https://holoviews.org/index.html)<br>
[Geoviews](https://geoviews.org/#) (Holoviews but for geographic data visualization)<br>

**The home for browser-based data viz in Python:**<br>
[HoloViz](https://holoviz.org/index.html)

#### Acknowledgements
Much of today's lecture is adapted/borrowed from the [hvPlot Tutorials](https://hvplot.holoviz.org/en/docs/latest/tutorials/index.html)

# A little about hvPlot and the HoloViz Project

In the simplest sense, hvPlot is a tool (really, it's an API, or application programming interface) you can use to make interactive plots. It works with a wide variety of data analysis packages that you are already familiar with, like pandas and xarray. Like seaborn, it is particuarly useful for data exploration, and it has a similar goal--to make plotting easier so that you spend less time trying to plot and more time thinking about your data. hvPlot is part of the [HoloViz](https://holoviz.org/index.html) which is "a coordinated effort to make browser-based data visualization in Python easier to use, easier to learn, and more powerful." Check out some of the HoloViz examples [here](https://examples.holoviz.org/gallery/index.html)!

Let's begin by importing the packages we'll use today that we're already familiar with:

In [ ]:
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt

---

## 1. How hvPlot "works"

hvPlot leverages the built-in plotting tools that come standard in many data analysis packages. You can control whether the output is interactive or static by choosing the package that hvPlot uses to render the plot itself (Bokeh, Plotly, or Matplotlib).

<img src='https://hvplot.holoviz.org/en/docs/latest/_images/diagram.svg' width=600></img>
<br><font size=1>Image Credit: [hvPlot](https://hvplot.holoviz.org/en/docs/latest/index.html#hvplot)</font>

**hvPlot can work with many different data analysis packages, so you need to tell it what package you are working with when you import it!** For the examples in this notebook we'll be using pandas and xarray, so we'll import `hvplot.pandas` and `hvplot.xarray`

In [ ]:
import hvplot.pandas
import hvplot.xarray

## 2. Setting the hvPlot plotting default
The default plotting output for hvPlot is an interactive plot built with [Bokeh](https://bokeh.org/). If you want to switch to using matplotlib (not interactive) or [plotly](https://plotly.com/python/) (interactive) then you can do so with the following commands: `hvplot.extension('matplotlib')` or `hvplot.extension('plotly')`. You can also load multiple extensions at once, **but the first will always be the default.** As an example, let's load all three extensions, with Bokeh as the default:

In [ ]:
hvplot.extension('bokeh','matplotlib','plotly')

## 3. Plotting with hvPlot
Using hvplot instead of any of the built-in plotting from xarray and pandas (or any other analysis package that hvplot works with) is straightforward. Let's read in our favorite Niño3.4 data for this example. We'll also clean everything up first to make it easier to work with.

In [ ]:
#Download data
!curl -O https://www.cpc.ncep.noaa.gov/products/analysis_monitoring/ensostuff/detrend.nino34.ascii.txt

#Read in data
nino=pd.read_csv('detrend.nino34.ascii.txt',sep='\s+')

#Make a datetime index and set it as the index for the dataset
nino=nino.rename(columns={'YR':'year','MON':'month'}) #rename syntax
new_index=pd.to_datetime(nino[['year', 'month']].assign(days=np.ones(len(nino.ANOM.values)))) 
nino=nino.set_index(new_index)

#print the first 5 lines
nino.head()

Now that we have our data, let's make a plot of the SST anomalies over time. Normally we'd just do that with the `.plot()` command, but to use hvplot instead, you just us `.hvplot()` instead

In [ ]:
nino['ANOM'].hvplot()

Cool! You can now use pan, zoom, and hover for all your plots for easier exploration. What happens if you want to just go back to a static matplotlib style plot? That's straightforward too. Just change the backend:

In [ ]:
hvplot.output(backend='matplotlib') #could be plotly or bokeh too
nino['ANOM'].hvplot()

Ok that's fine, but let's change out backend back to bokeh so that we can keep learning how to use interactive plots:

In [ ]:
hvplot.output(backend='bokeh')

### Supported plot types
hvplot supports [many different types of plots](https://hvplot.holoviz.org/en/docs/latest/user_guide/Plotting.html#the-hvplot-namespace) that you can specify using their actual names, or using the `kind` argument similar to seaborn. Let's use seaborn's penguins dataset for this example:

In [ ]:
import seaborn as sns
penguins = sns.load_dataset("penguins")
penguins.head()

Let's make a scatterplot of penguin flipper length vs body mass. You can do that in hvplot by specifying the plot type using the 'kind' argument or through the hvplot namespace:

In [ ]:
#Use the hvplot namescape directly
penguins.hvplot.scatter(x='body_mass_g',y='flipper_length_mm',color='darkmagenta')

#Use the 'kind' argument
#penguins.hvplot(x='body_mass_g',y='flipper_length_mm',kind='scatter')

Here's an example of how to make a [histogram](https://hvplot.holoviz.org/en/docs/latest/ref/api/manual/hvplot.hvPlot.hist.html) with two variables on the same plot:

In [ ]:
#Histogram with two variables
penguins.hvplot.hist(y=['bill_depth_mm', 'bill_length_mm'],color=['goldenrod','navy'])

You can also customize plots and make subplots, etc:
><b><font color='red'>CAUTION:</font></b> The specific keyword arguments that you use to customize a plot depend on the backend you are using to render the plots. Matplotlib will not take the exact same keywords as bokeh or plotly!

In [ ]:
#Histogram of Nino3.4
nino.hvplot.hist(y=['ANOM','TOTAL'],subplots=True,width=400,color='teal',line_color='k',shared_axes=False)

### The explorer interface + widgets

><b><font color='red'>Note:</font></b> Normally these things should live update in the notebook, but alas, none of them are playing super nice. The examples in this section are just meant to show you what's possible--you should be able to do these just fine in your own environments outside of the jupyterhub we are using.

What if you have a dataset with lots of different types of information, and you aren't even sure what plots to make? Use hvplot's [explorer interface](https://hvplot.holoviz.org/en/docs/latest/user_guide/Explorer.html) (new as of version 0.8.0). This allows you to make any of the plots in the hvplot namespace with any of the columns of your data.

In [ ]:
#Apply the explorer interface to the penguins dataset
penguins.hvplot.explorer()

Not every dataset you have is going to fit nicely into the explorer interface. Nevertheless, you can also build your own widgets for your datasets to make exploration a little easier. [Widgets](https://hvplot.holoviz.org/en/docs/latest/user_guide/Interactive.html#interactive-widgets) are used with the [interactive API](https://hvplot.holoviz.org/en/docs/latest/user_guide/Interactive.html), which itself lets you pass in a widget instead of one value, such as a single time, for example. That means you can look at plots through time without having to create all of them separately!

To do that we need to import a few things that help us make widgets:

In [ ]:
import ipywidgets as ipw
import panel as pn
import panel.widgets as pnw

Let's use our MERRA2 data as an example:

In [ ]:
#Read in the data
ds=xr.open_mfdataset('/share/Notebook_data/MERRA2/*.nc4')
ds

Plot a map of the data with a slider that lets you change the plot in time using Panel's Discrete Slider widget:

In [ ]:
ds.T2MMEAN.interactive.sel(time=pnw.DiscreteSlider).plot()

---